In [ ]:
# BLOCK A — Install each library separately (safe for Colab)
!!pip install ultralytics
!!pip install roboflow
!!pip install huggingface_hub
!!pip install tensorboard
!!pip install matplotlib
!!pip install pandas
!!pip install seaborn
!!pip install onnx
!!pip install onnxruntime

# OPTIONAL (only if you need TensorFlow for TFLite conversion — NOT needed for training)
!!!pip install "tensorflow>=2.12.0"


In [ ]:
import torch, os

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

# ---- training settings ----
RUN_NAME = "yolov11n_15K_run"
MODEL_SIZE = "yolov11n.pt"   # nano model
IMGSZ = 640
BATCH = 16
SESSION_EPOCHS = 20          # run 20 epochs per session
WORKERS = 4

# ultralytics will save runs here:
PROJECT_PATH = "/content/runs"
RUN_DIR = f"{PROJECT_PATH}/train/{RUN_NAME}"
LAST_PT = f"{RUN_DIR}/weights/last.pt"
BEST_PT = f"{RUN_DIR}/weights/best.pt"

print("Project path:", PROJECT_PATH)
print("Run directory:", RUN_DIR)


Torch version: 2.9.0+cu126
CUDA available: True
Mon Dec  1 05:40:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="")

project = rf.workspace("training-zjioa").project("merger-ax44t-4b1mr")
version = project.version(4)
dataset = version.download("yolov11")

DATA_DIR = dataset.location
DATA_YAML = DATA_DIR + "/data.yaml"

print("Dataset stored at:", DATA_DIR)
print("Using YAML:", DATA_YAML)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Merger-4 in yolov11:: 100%|██████████| 30685/30685 [00:05<00:00, 5847.29it/s]


Dataset stored at: /content/Merger-4
Using YAML: /content/Merger-4/data.yaml


In [ ]:
# BLOCK D (robust) - TRAINING with fallback pretrained checkpoint
from ultralytics import YOLO
import time, os

# MODEL_SIZE may be 'yolov11n.pt' (preferred) but we will fallback to yolov8n.pt
WANTED_MODEL = "yolov11n.pt"
FALLBACK_MODEL = "yolov8n.pt"  # guaranteed available from Ultralytics hub

# If a last.pt exists from previous run, always resume from it
if os.path.exists(LAST_PT):
    print("Resuming training from existing weights:", LAST_PT)
    model = YOLO(LAST_PT)
    resume_flag = True
else:
    # Try to load the preferred model file if it exists locally or is auto-downloadable
    try:
        print(f"Attempting to load preferred pretrained checkpoint: {WANTED_MODEL}")
        model = YOLO(WANTED_MODEL)   # will throw FileNotFoundError if not present
        print(f"Loaded {WANTED_MODEL} successfully.")
        resume_flag = False
    except FileNotFoundError:
        print(f"WARNING: {WANTED_MODEL} not found locally.")
        # Try fallback that Ultralytics will download automatically
        try:
            print(f"Falling back to {FALLBACK_MODEL} (will be auto-downloaded if necessary).")
            model = YOLO(FALLBACK_MODEL)
            print(f"Loaded fallback model {FALLBACK_MODEL}.")
            resume_flag = False
        except Exception as e:
            # If even fallback fails, surface helpful troubleshooting info
            print("ERROR: Could not load fallback model. Full exception:")
            raise e

print("\n🔧 Training this session for", SESSION_EPOCHS, "epochs (resume=", resume_flag, ")...\n")

t0 = time.time()
model.train(
    data=DATA_YAML,
    epochs=SESSION_EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    workers=WORKERS,
    device=0,
    project=PROJECT_PATH,
    name=RUN_NAME,
    exist_ok=True,
    resume=resume_flag
)
t1 = time.time()
print(f"\n⏳ Session training time: {(t1-t0)/60:.2f} minutes\n")


Attempting to load preferred pretrained checkpoint: yolov11n.pt
Falling back to yolov8n.pt (will be auto-downloaded if necessary).
Loaded fallback model yolov8n.pt.

🔧 Training this session for 20 epochs (resume= False )...

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Merger-4/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, 

In [ ]:
# === ONE BLOCK: Mount Drive, create folder, copy session weights ===

from google.colab import drive
import os, shutil

# 1) Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)
print("Drive mounted.")

# 2) Define run name (must match your training run folder in /content/runs)
RUN_NAME = "yolov11n_15K_run"   # change if your run name is different

# 3) Paths
LOCAL_WEIGHTS = f"/content/runs/{RUN_NAME}/weights"
DRIVE_ROOT = "/content/gdrive/MyDrive/YOLO_Training_15K"
DRIVE_RUN_DIR = f"{DRIVE_ROOT}/runs/train/{RUN_NAME}"
DRIVE_WEIGHTS = f"{DRIVE_RUN_DIR}/weights"

# 4) Create Drive folder
os.makedirs(DRIVE_WEIGHTS, exist_ok=True)
print("Created Drive folder:", DRIVE_WEIGHTS)

# 5) Copy best.pt
if os.path.exists(f"{LOCAL_WEIGHTS}/best.pt"):
    shutil.copy(f"{LOCAL_WEIGHTS}/best.pt", f"{DRIVE_WEIGHTS}/best.pt")
    print("✔ Copied best.pt to Drive.")
else:
    print("✘ best.pt not found in local training folder.")

# 6) Copy last.pt
if os.path.exists(f"{LOCAL_WEIGHTS}/last.pt"):
    shutil.copy(f"{LOCAL_WEIGHTS}/last.pt", f"{DRIVE_WEIGHTS}/last.pt")
    print("✔ Copied last.pt to Drive.")
else:
    print("✘ last.pt not found in local training folder.")

print("\n🎉 Session 1 weights successfully saved to Google Drive!")


Mounted at /content/gdrive
Drive mounted.
Created Drive folder: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights
✔ Copied best.pt to Drive.
✔ Copied last.pt to Drive.

🎉 Session 1 weights successfully saved to Google Drive!


In [ ]:
# === Complete evaluation block: safe metrics + per-class table + plot + CSV save ===
# Paste & run in Colab (assumes ultralytics installed and Drive mounted if you use Drive paths)

from IPython.display import clear_output, display
import os, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from ultralytics import YOLO

# ---------------- CONFIG (edit if needed) ----------------
RUN_NAME = "yolov11n_15K_run"                            # your run name
DRIVE_ROOT = "/content/gdrive/MyDrive/YOLO_Training_15K" # where you backed up runs
SESSION_RUN_DIR = f"/content/runs/{RUN_NAME}"           # session run dir
DRIVE_RUN_DIR   = os.path.join(DRIVE_ROOT, "runs", "train", RUN_NAME)  # drive run dir
DATA_YAML = "/content/Merger-4/data.yaml"               # path to your data.yaml
IMGSZ = 640
BATCH = 16
SAVE_PER_CLASS_CSV = os.path.join(DRIVE_RUN_DIR, "per_class_metrics.csv") # output CSV
# --------------------------------------------------------

clear_output(wait=True)
print("Starting evaluation — this will print a clean metrics report.\n")

# ---------- Find best.pt (prefer Drive) ----------
best_candidates = []
drive_best = os.path.join(DRIVE_RUN_DIR, "weights", "best.pt")
session_best = os.path.join(SESSION_RUN_DIR, "weights", "best.pt")
drive_last = os.path.join(DRIVE_RUN_DIR, "weights", "last.pt")
session_last = os.path.join(SESSION_RUN_DIR, "weights", "last.pt")

for p in (drive_best, session_best, drive_last, session_last):
    if os.path.exists(p):
        best_candidates.append(p)

if not best_candidates:
    raise FileNotFoundError("No best.pt or last.pt found in Drive or session run directories. "
                            "Make sure you've copied weights to Drive or that /content/runs contains weights.")

BEST_PT = best_candidates[0]
print("Using weights:", BEST_PT)
print()

# ---------------- Load model ----------------
model = YOLO(BEST_PT)

# ---------------- Evaluate ----------------
print("Running evaluation on VALIDATION split...")
val_results = model.val(data=DATA_YAML, split="val", imgsz=IMGSZ, batch=BATCH, device=0)

print("Running evaluation on TEST split...")
test_results = model.val(data=DATA_YAML, split="test", imgsz=IMGSZ, batch=BATCH, device=0)

# ---------------- Helpers ----------------
def scalarize(x):
    """Convert scalar/array/list -> float (mean) or None if invalid."""
    try:
        if x is None:
            return None
        if isinstance(x, (list, tuple)):
            x = np.array(x)
        if isinstance(x, np.ndarray):
            # empty array
            if x.size == 0:
                return None
            return float(np.nanmean(x))
        return float(x)
    except Exception:
        return None

def safe_get(obj, *attrs):
    """Try nested attribute retrieval: safe_get(obj, 'box','p') -> obj.box.p or None."""
    cur = obj
    for a in attrs:
        cur = getattr(cur, a, None)
        if cur is None:
            return None
    return cur

# ---------------- Global summary scalars ----------------
val_map50   = scalarize(safe_get(val_results, "box", "map50"))
val_map     = scalarize(safe_get(val_results, "box", "map"))
val_p_mean  = scalarize(safe_get(val_results, "box", "p"))
val_r_mean  = scalarize(safe_get(val_results, "box", "r"))

test_map50  = scalarize(safe_get(test_results, "box", "map50"))
test_map    = scalarize(safe_get(test_results, "box", "map"))
test_p_mean = scalarize(safe_get(test_results, "box", "p"))
test_r_mean = scalarize(safe_get(test_results, "box", "r"))

# Clear output and print clean summary
clear_output(wait=True)
print("=== WEIGHTS USED ===")
print(" ", BEST_PT, "\n")

print("=== VALIDATION SUMMARY ===")
print(f"mAP@50:     {val_map50:.4f}" if val_map50 is not None else "mAP@50:     N/A")
print(f"mAP@50-95:  {val_map:.4f}" if val_map is not None else "mAP@50-95:  N/A")
print(f"Precision (mean): {val_p_mean:.4f}" if val_p_mean is not None else "Precision (mean): N/A")
print(f"Recall    (mean): {val_r_mean:.4f}" if val_r_mean is not None else "Recall    (mean): N/A")
print()

print("=== TEST SUMMARY ===")
print(f"mAP@50:     {test_map50:.4f}" if test_map50 is not None else "mAP@50:     N/A")
print(f"mAP@50-95:  {test_map:.4f}" if test_map is not None else "mAP@50-95:  N/A")
print(f"Precision (mean): {test_p_mean:.4f}" if test_p_mean is not None else "Precision (mean): N/A")
print(f"Recall    (mean): {test_r_mean:.4f}" if test_r_mean is not None else "Recall    (mean): N/A")
print()

# ---------------- Per-class metrics table ----------------
# Try to get per-class arrays (AP, precision, recall)
# Common attributes: test_results.box.maps (AP per class), test_results.box.p (precision per class), .r (recall per class)
aps = safe_get(test_results, "box", "maps") or safe_get(test_results, "box", "ap") or safe_get(test_results, "box", "aps")
prec_per_class = safe_get(test_results, "box", "p")
rec_per_class = safe_get(test_results, "box", "r")
names_attr = getattr(test_results, "names", None)

# Normalize to numpy arrays if possible
def to_np(x):
    try:
        if x is None:
            return None
        if isinstance(x, (list, tuple)):
            return np.array(x, dtype=float)
        if isinstance(x, np.ndarray):
            return x.astype(float)
        # try to convert iterable
        return np.array(list(x), dtype=float)
    except Exception:
        return None

aps_arr = to_np(aps)
prec_arr = to_np(prec_per_class)
rec_arr = to_np(rec_per_class)

if aps_arr is None:
    print("Per-class AP not available from results object. Skipping per-class table.\n")
else:
    n_classes = int(aps_arr.shape[0])
    # build names list
    if isinstance(names_attr, dict):
        names_list = [names_attr.get(i, str(i)) for i in range(n_classes)]
    elif isinstance(names_attr, (list, tuple)):
        names_list = list(names_attr[:n_classes])
    else:
        names_list = [str(i) for i in range(n_classes)]

    # assemble dataframe
    data = []
    for i in range(n_classes):
        ap_i = float(aps_arr[i]) if (i < len(aps_arr)) else np.nan
        p_i  = float(prec_arr[i]) if (prec_arr is not None and i < len(prec_arr)) else np.nan
        r_i  = float(rec_arr[i]) if (rec_arr is not None and i < len(rec_arr)) else np.nan
        data.append((i, names_list[i], ap_i, p_i, r_i))
    df_per_class = pd.DataFrame(data, columns=["class_id","class_name","AP50-95","Precision","Recall"])
    df_per_class = df_per_class.sort_values("AP50-95", ascending=False).reset_index(drop=True)

    # print pretty table (top & bottom)
    pd.set_option('display.precision', 4)
    print("=== PER-CLASS METRICS (TEST) — top 10 shown ===")
    display(df_per_class.head(10))
    print("... (full table saved to Drive at)", SAVE_PER_CLASS_CSV, "\n")

    # Save CSV to Drive run folder
    try:
        os.makedirs(os.path.dirname(SAVE_PER_CLASS_CSV), exist_ok=True)
        df_per_class.to_csv(SAVE_PER_CLASS_CSV, index=False)
    except Exception as e:
        print("Warning: could not save per-class CSV to Drive:", e)

    # ---------------- Plot per-class AP bar chart ----------------
    try:
        plt.figure(figsize=(10, max(4, n_classes*0.25)))
        plt.barh(df_per_class['class_name'][::-1], df_per_class['AP50-95'][::-1])
        plt.xlabel("AP (50-95) — test")
        plt.title(f"Per-class AP (test) — {RUN_NAME}")
        plt.grid(axis='x', linestyle='--', alpha=0.4)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print("Plotting failed:", e)

print("\n✅ Evaluation complete. If you want additional plots (PR curves, confusion matrix, loss curves) say 'plots please'.")


=== WEIGHTS USED ===
  /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/best.pt 

=== VALIDATION SUMMARY ===
mAP@50:     0.7258
mAP@50-95:  0.5137
Precision (mean): 0.7742
Recall    (mean): 0.6828

=== TEST SUMMARY ===
mAP@50:     0.7344
mAP@50-95:  0.5029
Precision (mean): 0.8866
Recall    (mean): 0.6396



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# === Robust Roboflow deploy block: sanitizes model_name and handles ultralytics version switch ===
import os, re, subprocess, sys, time, json
from roboflow import Roboflow

# ----------------- EDIT THESE -----------------
RF_API_KEY = ""      # your Roboflow API key
WORKSPACE = "training-zjioa"
PROJECT_ID = "merger-ax44t-4b1mr"
DEPLOY_DIR = "/content/yolo_export_rf"   # folder you already created (contains model.pt, data.yaml, classes.txt)
REQUESTED_MODEL_NAME = "epoch20model"   # change as you like; this block will sanitize it
AUTO_SWITCH_ULTRALYTICS = True           # set False if you do not want !pip changes
REQUIRED_ULTRALYTICS_VER = "8.0.196"     # Roboflow compatibility target
# ----------------------------------------------

def sanitize_model_name(name: str) -> str:
    # replace spaces with dash, keep letters/numbers/dash, ensure at least one letter, <50 chars
    s = name.strip()
    s = s.replace(" ", "-")
    # remove invalid chars, allow letters, numbers, dashes, underscores
    s = re.sub(r"[^A-Za-z0-9\-_]+", "", s)
    if len(s) == 0:
        s = "model"
    # ensure there's at least one letter; if not, append 'm'
    if re.search(r"[A-Za-z]", s) is None:
        s = "m" + s
    # ensure length < 50
    if len(s) > 49:
        s = s[:49]
    return s

MODEL_NAME = sanitize_model_name(REQUESTED_MODEL_NAME)
print("Sanitized model_name ->", MODEL_NAME)

# Verify deploy folder exists and contains model.pt
if not os.path.isdir(DEPLOY_DIR):
    raise FileNotFoundError(f"Deploy folder not found: {DEPLOY_DIR}")

model_pt = os.path.join(DEPLOY_DIR, "model.pt")
if not os.path.exists(model_pt):
    raise FileNotFoundError(f"model.pt not found in {DEPLOY_DIR}")

# Optional: capture current ultralytics version
def !pip_install(package):
    print("Running !pip:", package)
    rc = subprocess.run([sys.executable, "-m", "!pip", "install", package, "--quiet"], capture_output=True, text=True)
    if rc.returncode != 0:
        print("!pip install failed:", rc.stderr[:1000])
        raise RuntimeError("!pip install failed")
    print("!pip install succeeded")

orig_ultralytics_ver = None
if AUTO_SWITCH_ULTRALYTICS:
    try:
        import ultralytics as ul
        orig_ultralytics_ver = getattr(ul, "__version__", None)
        print("Current ultralytics version:", orig_ultralytics_ver)
    except Exception:
        orig_ultralytics_ver = None
    # If different than required, install required
    if orig_ultralytics_ver != REQUIRED_ULTRALYTICS_VER:
        print("Installing required ultralytics==%s for Roboflow deploy..." % REQUIRED_ULTRALYTICS_VER)
        !pip_install(f"ultralytics=={REQUIRED_ULTRALYTICS_VER}")
        # re-import to be safe
        try:
            import importlib
            importlib.reload(__import__("ultralytics"))
        except Exception:
            pass
        time.sleep(1)

# Connect to Roboflow and attempt deployment
rf = Roboflow(api_key=RF_API_KEY)
workspace = rf.workspace(WORKSPACE)

print("Uploading/deploying model to Roboflow workspace:", WORKSPACE)
try:
    deploy_result = workspace.deploy_model(
        model_type="yolov8",
        model_path=DEPLOY_DIR,          # must be a directory (we prepared it)
        filename="model.pt",
        project_ids=[PROJECT_ID],
        model_name=MODEL_NAME
    )
    print("Deployment result (raw):")
    print(deploy_result)
except Exception as e:
    # print full exception and try to show HTTP details if present
    print("Deployment ERROR:", repr(e))
    try:
        import traceback
        traceback.print_exc()
    except Exception:
        pass

# Restore original ultralytics if we changed it
if AUTO_SWITCH_ULTRALYTICS and orig_ultralytics_ver is not None:
    try:
        import ultralytics as ul2
        cur_ver = getattr(ul2, "__version__", None)
    except Exception:
        cur_ver = None
    if cur_ver != orig_ultralytics_ver:
        print("Restoring original ultralytics version:", orig_ultralytics_ver)
        !pip_install(f"ultralytics=={orig_ultralytics_ver}")
        # re-import
        try:
            import importlib
            importlib.reload(__import__("ultralytics"))
        except Exception:
            pass

print("Done. If Roboflow still returned an error, inspect the printed exception above.")


Sanitized model_name -> epoch20model
Current ultralytics version: 8.3.233
Installing required ultralytics==8.0.196 for Roboflow deploy...
Running pip: ultralytics==8.0.196
pip install succeeded
loading Roboflow workspace...
Uploading/deploying model to Roboflow workspace: training-zjioa
View the status of your deployment for project merger-ax44t-4b1mr at: https://app.roboflow.com/training-zjioa/merger-ax44t-4b1mr/models
Deployment result (raw):
None
Restoring original ultralytics version: 8.3.233
Running pip: ultralytics==8.3.233
pip install succeeded
Done. If Roboflow still returned an error, inspect the printed exception above.


Re run

In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install huggingface_hub
!pip install tensorboard
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install onnx
!pip install onnxruntime


In [ ]:
from google.colab import drive
import os

RUN_NAME = "yolov11n_15K_run"
DRIVE_ROOT = "/content/gdrive/MyDrive/YOLO_Training_15K"
DRIVE_RUN_DIR = os.path.join(DRIVE_ROOT, "runs", "train", RUN_NAME)

drive.mount('/content/gdrive', force_remount=True)

print("Drive mounted.")
print("Run directory:", DRIVE_RUN_DIR)

LAST_PT = os.path.join(DRIVE_RUN_DIR, "weights", "last.pt")
BEST_PT = os.path.join(DRIVE_RUN_DIR, "weights", "best.pt")

print("last.pt:", LAST_PT)
print("best.pt:", BEST_PT)


Mounted at /content/gdrive
Drive mounted.
Run directory: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run
last.pt: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/last.pt
best.pt: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/best.pt


In [ ]:
# BLOCK C - Roboflow download (run once, optional if dataset already present)
from roboflow import Roboflow
import os, shutil

RF_API_KEY = ""  # <-- put your API key here
RF_WORKSPACE = "training-zjioa"
RF_PROJECT = "merger-ax44t-4b1mr"
RF_VERSION = 4
RF_FORMAT = "yolov11"   # format exported



rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace(RF_WORKSPACE).project(RF_PROJECT)
version = project.version(RF_VERSION)
dataset = version.download(RF_FORMAT)

DATA_DIR = dataset.location
DATA_YAML = os.path.join(DATA_DIR, "data.yaml")
print("Dataset downloaded to:", DATA_DIR)
print("DATA_YAML path:", DATA_YAML)

# Optional: copy data.yaml into Drive for safekeeping
try:
    dst = os.path.join(DRIVE_ROOT, "data.yaml")
    shutil.copy(DATA_YAML, dst)
    print("Copied data.yaml to Drive:", dst)
except Exception:
    pass


loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: /content/Merger-4
DATA_YAML path: /content/Merger-4/data.yaml
Copied data.yaml to Drive: /content/gdrive/MyDrive/YOLO_Training_15K/data.yaml


In [ ]:
# === RESUME TRAINING CELL (run this to continue training for the next SESSION_EPOCHS) ===
# Paste into Colab and run.

# NOTE: This starts a NEW training session from your last weights (resume=False).
# It will continue training from the checkpoint (weights content), adding SESSION_EPOCHS more.

from google.colab import drive
from ultralytics import YOLO
import torch, os, time, csv, numpy as np

# ---- USER CONFIG ----
RUN_NAME = "yolov11n_15K_run"                        # your run name
DRIVE_ROOT = "/content/gdrive/MyDrive/YOLO_Training_15K"
DATA_YAML = "/content/Merger-4/data.yaml"            # path to your dataset yaml in runtime
IMGSZ = 640
BATCH = 16
SESSION_EPOCHS = 20    # how many additional epochs to run in this session (e.g. 20 -> epochs 21-40)
WORKERS = 4
# ---------------------

# Mount Drive
drive.mount('/content/gdrive', force_remount=True)
print("Drive mounted:", DRIVE_ROOT)

# Paths
PROJECT_PATH = os.path.join(DRIVE_ROOT, "runs")                    # where Ultralytics will save
RUN_DIR      = os.path.join(PROJECT_PATH, "train", RUN_NAME)
LAST_PT      = os.path.join(RUN_DIR, "weights", "last.pt")
BEST_PT      = os.path.join(RUN_DIR, "weights", "best.pt")
METRICS_CSV  = os.path.join(DRIVE_ROOT, "session_metrics.csv")

os.makedirs(PROJECT_PATH, exist_ok=True)
os.makedirs(os.path.join(RUN_DIR, "weights"), exist_ok=True)

print("Project path (Drive):", PROJECT_PATH)
print("Run dir:", RUN_DIR)
print("Looking for LAST_PT:", LAST_PT)
print("Looking for BEST_PT:", BEST_PT)
print("CUDA available:", torch.cuda.is_available())

# Choose weights to continue from
weights_to_load = None
if os.path.exists(LAST_PT):
    weights_to_load = LAST_PT
    print("Found last.pt -> continuing from LAST_PT")
elif os.path.exists(BEST_PT):
    weights_to_load = BEST_PT
    print("last.pt not found; found best.pt -> continuing from BEST_PT")
else:
    print("No last.pt or best.pt found in Drive run dir. Will try to load preferred pretrained model and start fresh.")
    weights_to_load = None

# Load model: if weights_to_load present, use it; otherwise try preferred checkpoint or fallback
MODEL_PREFERRED = "yolov11n.pt"
MODEL_FALLBACK  = "yolov8n.pt"

if weights_to_load:
    model = YOLO(weights_to_load)
else:
    try:
        model = YOLO(MODEL_PREFERRED)
        print("Loaded preferred pretrained:", MODEL_PREFERRED)
    except Exception as e:
        print("Preferred not available, falling back to:", MODEL_FALLBACK, " error:", e)
        model = YOLO(MODEL_FALLBACK)

# Train: start a NEW session of SESSION_EPOCHS (do NOT use resume=True)
print(f"\nStarting NEW training session for {SESSION_EPOCHS} epochs (this will append to existing progress).")
t0 = time.time()
model.train(
    data=DATA_YAML,
    epochs=SESSION_EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=WORKERS,
    project=PROJECT_PATH,   # IMPORTANT: saves into Drive
    name=RUN_NAME,
    exist_ok=True,
    resume=False           # <<-- MUST BE False to avoid "nothing to resume" assert
)
t1 = time.time()
print(f"\nSession finished in {(t1-t0)/60:.2f} minutes. Check Drive: {RUN_DIR}")

# ---------- Quick evaluation on VALID and TEST and append metrics to Drive CSV ----------
print("\nRunning evaluation on VALID and TEST splits... (this may take a few minutes)")

# load latest best or last weights for evaluation
eval_weights = BEST_PT if os.path.exists(BEST_PT) else LAST_PT if os.path.exists(LAST_PT) else None
if eval_weights is None:
    print("No best or last weights found for evaluation, skipping evaluation.")
else:
    eval_model = YOLO(eval_weights)
    val_res = eval_model.val(data=DATA_YAML, split="val", imgsz=IMGSZ, batch=BATCH, device=0)
    test_res = eval_model.val(data=DATA_YAML, split="test", imgsz=IMGSZ, batch=BATCH, device=0)

    # helper scalarizer
    def scalarize(x):
        try:
            if x is None: return None
            if isinstance(x, (list, tuple)): x = np.array(x)
            if isinstance(x, np.ndarray):
                if x.size==0: return None
                return float(np.nanmean(x))
            return float(x)
        except Exception:
            return None

    val_map50 = scalarize(getattr(val_res.box, "map50", None))
    val_map    = scalarize(getattr(val_res.box, "map", None))
    val_p      = scalarize(getattr(val_res.box, "p", None))
    val_r      = scalarize(getattr(val_res.box, "r", None))

    test_map50 = scalarize(getattr(test_res.box, "map50", None))
    test_map   = scalarize(getattr(test_res.box, "map", None))
    test_p     = scalarize(getattr(test_res.box, "p", None))
    test_r     = scalarize(getattr(test_res.box, "r", None))

    # Print concise summary
    print("\n=== EVALUATION SUMMARY ===")
    print("Eval weights:", eval_weights)
    print(f"VAL  mAP@50: {val_map50}   mAP@50-95: {val_map}   P:{val_p}   R:{val_r}")
    print(f"TEST mAP@50: {test_map50}   mAP@50-95: {test_map}   P:{test_p}   R:{test_r}")

    # Append to CSV
    header = ["timestamp","weights_file","val_map50","val_map50-95","val_precision","val_recall",
              "test_map50","test_map50-95","test_precision","test_recall"]
    from datetime import datetime
    if not os.path.exists(METRICS_CSV):
        with open(METRICS_CSV, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)

    with open(METRICS_CSV, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([datetime.utcnow().isoformat(), os.path.basename(eval_weights),
                         val_map50, val_map, val_p, val_r,
                         test_map50, test_map, test_p, test_r])
    print("Appended evaluation metrics to:", METRICS_CSV)

print("\n✅ Resume complete. Monitor training logs in Drive or here. If you want, run the evaluation/plot block next.")


Mounted at /content/gdrive
Drive mounted: /content/gdrive/MyDrive/YOLO_Training_15K
Project path (Drive): /content/gdrive/MyDrive/YOLO_Training_15K/runs
Run dir: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run
Looking for LAST_PT: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/last.pt
Looking for BEST_PT: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/best.pt
CUDA available: True
Found last.pt -> continuing from LAST_PT

Starting NEW training session for 20 epochs (this will append to existing progress).
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Merger-4/data.yaml, degrees=0.0, determin

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).


In [ ]:
# BLOCK E - Evaluate on val & test and append metrics to CSV on Drive
from datetime import datetime
import csv, glob, pandas as pd, os
from ultralytics import YOLO

# ensure run dir targets the Drive location
RUN_DIR = os.path.join(DRIVE_ROOT, "runs", "train", RUN_NAME)
METRICS_CSV = os.path.join(DRIVE_ROOT, "session_metrics.csv")

# load best model (prefer Drive best)
best = os.path.join(RUN_DIR, "weights", "best.pt")
last = os.path.join(RUN_DIR, "weights", "last.pt")
weights_to_use = best if os.path.exists(best) else last
if not weights_to_use or not os.path.exists(weights_to_use):
    raise FileNotFoundError("No best.pt or last.pt found in Drive run dir. Check TRAIN saved files.")

print("Evaluating using weights:", weights_to_use)
model = YOLO(weights_to_use)

# Evaluate
val_res = model.val(data=DATA_YAML, split="val", imgsz=IMGSZ, batch=BATCH, device=0)
test_res = model.val(data=DATA_YAML, split="test", imgsz=IMGSZ, batch=BATCH, device=0)

# robust extractor
def scalarize(x):
    import numpy as _np
    try:
        if x is None: return None
        if isinstance(x, (list, tuple)): x = _np.array(x)
        if isinstance(x, _np.ndarray):
            if x.size==0: return None
            return float(_np.nanmean(x))
        return float(x)
    except Exception:
        return None

val_map50 = scalarize(getattr(val_res.box, "map50", None))
val_prec  = scalarize(getattr(val_res.box, "p", None))
val_rec   = scalarize(getattr(val_res.box, "r", None))

test_map50 = scalarize(getattr(test_res.box, "map50", None))
test_prec  = scalarize(getattr(test_res.box, "p", None))
test_rec   = scalarize(getattr(test_res.box, "r", None))

# Append CSV (create header if missing)
if not os.path.exists(METRICS_CSV):
    with open(METRICS_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp","weights","val_map50","val_precision","val_recall","test_map50","test_precision","test_recall"])

with open(METRICS_CSV, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([datetime.utcnow().isoformat(), os.path.basename(weights_to_use),
                     val_map50, val_prec, val_rec, test_map50, test_prec, test_rec])

print("Metrics appended to:", METRICS_CSV)
print("Validation mAP@50:", val_map50, " Test mAP@50:", test_map50)


Evaluating using weights: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/best.pt
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1552.7±419.0 MB/s, size: 54.7 KB)
val: Scanning /content/Merger-4/valid/labels.cache... 1682 images, 45 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1682/1682 2.9Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 106/106 6.4it/s 16.6s
                   all       1682       3855      0.774      0.683      0.726      0.514
                   Cow        407       1883      0.792      0.587      0.716      0.453
                Coyote         51         53      0.785      0.965      0.957      0.818
                  Deer        542        736      0.938      0.909      0.962      0.737
                  G

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).


In [ ]:
# === Evaluate test.pt from Drive WITHOUT overwriting best.pt ===
# Paste & run in Colab. Assumes Drive already mounted (if not, it will mount).

from google.colab import drive
from ultralytics import YOLO
from datetime import datetime
import shutil, os, csv, glob, numpy as np

# ---- USER SETTINGS ----
DRIVE_ROOT = "/content/gdrive/MyDrive/YOLO_Training_15K"  # your Drive project root
RUN_NAME = "yolov11n_15K_run"                             # your run name (unchanged)
MODEL_NAME = "best.pt"   # name of the model file in DRIVE_ROOT you want to eval (change if needed)
TMP_EVAL_PATH = "/content/eval_test.pt"  # temporary path in runtime (won't overwrite Drive files)

IMGSZ = 640
BATCH = 16
DATA_YAML_CANDIDATES = [
    "/content/Merger-4/data.yaml",                           # common session location
    os.path.join("/content", "data.yaml"),
    os.path.join(DRIVE_ROOT, "data.yaml"),
]
METRICS_CSV = os.path.join(DRIVE_ROOT, "session_metrics.csv")
# -----------------------

# Mount Drive (safe no-op if already mounted)
drive.mount('/content/gdrive', force_remount=False)

# Locate the model file in Drive
drive_model_path = os.path.join(DRIVE_ROOT, MODEL_NAME)
if not os.path.exists(drive_model_path):
    # try searching Drive root for the file
    matches = glob.glob(os.path.join(DRIVE_ROOT, "**", MODEL_NAME), recursive=True)
    if matches:
        drive_model_path = matches[0]
    else:
        raise FileNotFoundError(f"Could not find {MODEL_NAME} under {DRIVE_ROOT}.")

print("Found model in Drive at:", drive_model_path)

# Copy to runtime temp path to avoid touching Drive-run folders
shutil.copy2(drive_model_path, TMP_EVAL_PATH)
print("Copied to temporary runtime file:", TMP_EVAL_PATH)

# Discover DATA_YAML
DATA_YAML = None
for cand in DATA_YAML_CANDIDATES:
    if os.path.exists(cand):
        DATA_YAML = cand
        break
# If not found, try to find any data.yaml under /content or Drive run
if DATA_YAML is None:
    search = glob.glob("/content/**/data.yaml", recursive=True) + glob.glob(os.path.join(DRIVE_ROOT, "**", "data.yaml"), recursive=True)
    if search:
        DATA_YAML = search[0]

if DATA_YAML is None:
    raise FileNotFoundError("Could not locate data.yaml. Set DATA_YAML variable to your dataset yaml path.")

print("Using DATA_YAML:", DATA_YAML)

# Load model from tmp path and evaluate
print("\nLoading model for evaluation from:", TMP_EVAL_PATH)
model = YOLO(TMP_EVAL_PATH)

print("\nRunning validation (val split)...")
val_res = model.val(data=DATA_YAML, split="val", imgsz=IMGSZ, batch=BATCH, device=0)

print("\nRunning test (test split)...")
test_res = model.val(data=DATA_YAML, split="test", imgsz=IMGSZ, batch=BATCH, device=0)

# Helper to reduce arrays/lists to scalar mean
def scalarize(x):
    try:
        if x is None: return None
        if isinstance(x, (list, tuple)): x = np.array(x)
        if isinstance(x, np.ndarray):
            if x.size == 0: return None
            return float(np.nanmean(x))
        return float(x)
    except Exception:
        return None

val_map50 = scalarize(getattr(val_res.box, "map50", None))
val_map   = scalarize(getattr(val_res.box, "map", None))
val_p     = scalarize(getattr(val_res.box, "p", None))
val_r     = scalarize(getattr(val_res.box, "r", None))

test_map50 = scalarize(getattr(test_res.box, "map50", None))
test_map   = scalarize(getattr(test_res.box, "map", None))
test_p     = scalarize(getattr(test_res.box, "p", None))
test_r     = scalarize(getattr(test_res.box, "r", None))

# Print results
clear_header = "\n" + ("="*12) + " EVALUATION RESULTS " + ("="*12)
print(clear_header)
print("Model (Drive):", drive_model_path)
print("Model (temp):", TMP_EVAL_PATH)
print("\nVALIDATION:")
print(f"  mAP@50:      {val_map50}")
print(f"  mAP@50-95:   {val_map}")
print(f"  Precision:   {val_p}")
print(f"  Recall:      {val_r}")
print("\nTEST:")
print(f"  mAP@50:      {test_map50}")
print(f"  mAP@50-95:   {test_map}")
print(f"  Precision:   {test_p}")
print(f"  Recall:      {test_r}")
print("="*44 + "\n")

# Append to Drive CSV (create header if missing)
if not os.path.exists(METRICS_CSV):
    with open(METRICS_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp","model_file","val_map50","val_map50-95","val_precision","val_recall",
                         "test_map50","test_map50-95","test_precision","test_recall"])

with open(METRICS_CSV, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([datetime.utcnow().isoformat(), MODEL_NAME,
                     val_map50, val_map, val_p, val_r,
                     test_map50, test_map, test_p, test_r])

print("Appended evaluation metrics to:", METRICS_CSV)
print("Drive best.pt and other run files were NOT modified.")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found model in Drive at: /content/gdrive/MyDrive/YOLO_Training_15K/runs/train/yolov11n_15K_run/weights/best.pt
Copied to temporary runtime file: /content/eval_test.pt
Using DATA_YAML: /content/Merger-4/data.yaml

Loading model for evaluation from: /content/eval_test.pt

Running validation (val split)...
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1518.0±231.6 MB/s, size: 74.4 KB)
val: Scanning /content/Merger-4/valid/labels.cache... 1682 images, 45 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1682/1682 3.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 106/106 6.1it/s 17.3s
                   all       1682       3855      0

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
